In [1]:
pip install imblearn

In [2]:
pip install mlxtend

In [3]:
import numpy as np
import pandas as pd
import re, string
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.ticker as ticker
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE

In [4]:
df = pd.read_csv ('train_all_tasks.csv')
df.head (10)

,rewire_id,text,label_sexist,label_category,label_vector
0,sexism2022_english-7358,"Damn, this writing was pretty chaotic",not sexist,none,none
1,sexism2022_english-2367,"Yeah, and apparently a bunch of misogynistic v...",not sexist,none,none
2,sexism2022_english-3073,How the FUCK is this woman still an MP!!!???,not sexist,none,none
3,sexism2022_english-14895,Understand. Know you're right. At same time I ...,not sexist,none,none
4,sexism2022_english-4118,Surprized they didn't stop and rape some women,not sexist,none,none
5,sexism2022_english-10376,yeah but in order to keep the benefit i have t...,not sexist,none,none
6,sexism2022_english-4569,Hahaha I just wish they would live their truth...,not sexist,none,none
7,sexism2022_english-91,head mod said she talked to him after he made ...,not sexist,none,none
8,sexism2022_english-10142,"just the inside of your wallet: cash, bank car...",sexist,2. derogation,2.3 dehumanising attacks & overt sexual object...
9,sexism2022_english-16316,We most definitely did. I didn’t go too much i...,not sexist,none,none


In [5]:
def clean_text():
    clean = re.compile('<.*?>')

    df['text'] = df['text'].apply(lambda x: x.lower())
    df['text'] = df['text'].apply((lambda x: re.sub(clean, ' ', x)))
    df['text'] = df['text'].apply((lambda x: re.sub('https://.*', ' ', x)))

clean_text()


def drop_column(columns):
    global df
    df = df.drop(columns, axis = 1)

def drop_rows(columns):
    global df
    df = df.dropna(axis = 0, subset = columns)


drop = ['rewire_id']
drop_column(drop)

print( '\nThe shape of this dataset is now: {}\n'.format( df.shape ) )
df.tail()


The shape of this dataset is now: (14000, 4)



,text,label_sexist,label_category,label_vector
13995,complexes like the 'nice chicks' that go after...,sexist,2. derogation,2.1 descriptive attacks
13996,"""graphic germany - muslim ""refugee"" stabbing h...",not sexist,none,none
13997,lol i imagine there would be simps that are li...,not sexist,none,none
13998,"it's not, the girls i go on dates with don't k...",not sexist,none,none
13999,how can he be an incel? he does have a girlfri...,not sexist,none,none


In [6]:
label_binary = {
    'not sexist': 0,
    'sexist': 1,
    }

label_categorical = {
    'none': 0,
    '1. threats, plans to harm and incitement': 1,
    '2. derogation': 2,
    '3. animosity': 3,
    '4. prejudiced discussions': 4,
    }


df ['binary_sexism'] = df ['label_sexist'].apply( lambda x : label_binary [x] )
df ['category_sexism'] = df ['label_category'].apply( lambda x : label_categorical [x] )

df.tail()

,text,label_sexist,label_category,label_vector,binary_sexism,category_sexism
13995,complexes like the 'nice chicks' that go after...,sexist,2. derogation,2.1 descriptive attacks,1,2
13996,"""graphic germany - muslim ""refugee"" stabbing h...",not sexist,none,none,0,0
13997,lol i imagine there would be simps that are li...,not sexist,none,none,0,0
13998,"it's not, the girls i go on dates with don't k...",not sexist,none,none,0,0
13999,how can he be an incel? he does have a girlfri...,not sexist,none,none,0,0


In [7]:
x = df['text']
y = df['binary_sexism']

x_train, x_hold, y_train, y_hold = train_test_split (x, y, random_state = 42, test_size = 0.20, shuffle=True, stratify=y)
x_test, x_val, y_test, y_val = train_test_split (x_hold, y_hold, test_size = 0.50, random_state = 42, shuffle= True, stratify = y_hold)

In [8]:
num_words = 3000


tokenizer = Tokenizer (num_words = num_words, lower = False)


def tokenization( x_train, x_val, x_test ):

  tokenizer.fit_on_texts (x_train)

  tokenized_xtrain = tokenizer.texts_to_sequences (x_train)
  tokenized_xval = tokenizer.texts_to_sequences (x_val)
  tokenized_xtest = tokenizer.texts_to_sequences (x_test)

  return tokenized_xtrain, tokenized_xval, tokenized_xtest

x_train, x_val, x_test = tokenization (x_train, x_val, x_test)


vocab = tokenizer.word_index
vocab_size = len( tokenizer.word_index ) + 1

In [9]:
def padding (x_train, x_val, x_test):
    maxlen = 50

    padded_xtrain = pad_sequences( x_train, padding = 'post', maxlen = maxlen )

    #   maxlen = len( padded_xtrain[0] )
    padded_xval = pad_sequences( x_val, padding = 'post', maxlen = maxlen )
    padded_xtest = pad_sequences( x_test, padding = 'post', maxlen = maxlen )

    return maxlen, padded_xtrain, padded_xval, padded_xtest

maxlen, x_train, x_val, x_test = padding( x_train, x_val, x_test )


print("max length of each padded texts: ", maxlen)

print( "\n[ x_train.shape[0], x_train.shape[1] ]: [ {}, {} ]".format ( x_train.shape[0], x_train.shape[1] ) )
print( "\n[ x_val.shape[0], x_val.shape[1] ]: [ {}, {} ]".format ( x_val.shape[0], x_val.shape[1] ) )
print( "\n[ x_test.shape[0], x_test.shape[1] ]: [ {}, {} ]\n".format ( x_test.shape[0], x_test.shape[1] ) )

print( "X-train row index 3:\n", x_train[3, :] )

max length of each padded texts:  50

[ x_train.shape[0], x_train.shape[1] ]: [ 11200, 50 ]

[ x_val.shape[0], x_val.shape[1] ]: [ 1400, 50 ]

[ x_test.shape[0], x_test.shape[1] ]: [ 1400, 50 ]

X-train row index 3:
 [ 434  247  601   27   15   56    1   85  356  377  102   66   32  134
   48    3    1  146 1819 1820    4  954   34    7  357   21   19  233
 2241 1708    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [10]:
def create_embedding_matrix(filepath, word_index, embedding_dim):

    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix


embedding_dim = 100
embedding_matrix = create_embedding_matrix( 'glove.6B.100d.txt', tokenizer.word_index, embedding_dim )

print("The embedding matrix with GLoVe embedding: \n", embedding_matrix)


#Non zero embedding vectors

nonzero_elements = np.count_nonzero( np.count_nonzero(embedding_matrix, axis=1) )
print( "Amount of nonzero embedding vectors: ", nonzero_elements / vocab_size )

The embedding matrix with GLoVe embedding: 
 [[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.038194   -0.24487001  0.72812003 ... -0.1459      0.82779998
   0.27061999]
 [-0.27085999  0.044006   -0.02026    ... -0.4923      0.63687003
   0.23642001]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.40079001 -0.57675999  0.06379    ...  0.33825001 -0.037264
  -0.050584  ]]
Amount of nonzero embedding vectors:  0.3271790253177224


In [ ]:
def lstm_simple (dense, maxlen):
    lstm_out = 128

    model = Sequential()
    model.add( layers.Embedding( vocab_size, embedding_dim, weights = [embedding_matrix], input_length = maxlen, trainable = True ) )
    model.add( layers.LSTM ( lstm_out, dropout = 0.2 ) )
    model.add( layers.Dense ( dense, activation ='sigmoid' ) )

    model.compile ( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'] )
    print ( model.summary() )

    return model

model = lstm_simple (1, maxlen)

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 50, 100)           1998600   
                                                                 
 lstm_6 (LSTM)               (None, 128)               117248    
                                                                 
 dense_9 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2115977 (8.07 MB)
Trainable params: 2115977 (8.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
batch_size = 32
history = model.fit( x_train, y_train, epochs = 10, verbose = True, validation_data = ( x_val, y_val ), batch_size = batch_size )

Epoch 1/20
350/350 [==============================] - 71s 179ms/step - loss: 0.5524 - accuracy: 0.7562 - val_loss: 0.5524 - val_accuracy: 0.7571
Epoch 2/20
350/350 [==============================] - 48s 137ms/step - loss: 0.5481 - accuracy: 0.7570 - val_loss: 0.5339 - val_accuracy: 0.7579
Epoch 3/20
350/350 [==============================] - 46s 132ms/step - loss: 0.5415 - accuracy: 0.7555 - val_loss: 0.4973 - val_accuracy: 0.7836
Epoch 4/20
350/350 [==============================] - 46s 131ms/step - loss: 0.4696 - accuracy: 0.8122 - val_loss: 0.4487 - val_accuracy: 0.8264
Epoch 5/20
350/350 [==============================] - 47s 134ms/step - loss: 0.4123 - accuracy: 0.8274 - val_loss: 0.4325 - val_accuracy: 0.8207
Epoch 6/20
350/350 [==============================] - 43s 124ms/step - loss: 0.3605 - accuracy: 0.8490 - val_loss: 0.3954 - val_accuracy: 0.8343
Epoch 7/20
350/350 [==============================] - 47s 134ms/step - loss: 0.3244 - accuracy: 0.8629 - val_loss: 0.3974 - val_ac

In [ ]:
loss, accuracy = model.evaluate( x_train, y_train, verbose = False )
print( "\nTraining Accuracy: {:.4f}, Training Loss: {:.4f}".format(accuracy, loss) )

loss, accuracy = model.evaluate( x_test, y_test, verbose = False )
print( "Testing Accuracy:  {:.4f}, Testing Loss: {:.4f}\n".format(accuracy, loss) )

# print(history.history)


Training Accuracy: 0.9762, Training Loss: 0.0820
Testing Accuracy:  0.8150, Testing Loss: 0.6440



In [13]:
def lstm_bi (dense, maxlen):
    lstm_out = 128

    model1 = Sequential()
    model1.add( layers.Embedding( vocab_size, embedding_dim, weights = [embedding_matrix], input_length = maxlen, trainable = True ) )
    model1.add(layers.Bidirectional(layers.LSTM(lstm_out, dropout=0.2)))
    model1.add( layers.Dense ( dense, activation ='sigmoid' ) )

    model1.compile ( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'] )
    print ( model1.summary() )

    return model1

model1 = lstm_bi (1, maxlen)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 100)           1998600   
                                                                 
 bidirectional_1 (Bidirecti  (None, 256)               234496    
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 2233353 (8.52 MB)
Trainable params: 2233353 (8.52 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [14]:
batch_size = 32
history = model1.fit( x_train, y_train, epochs = 10, verbose = True, validation_data = ( x_val, y_val ), batch_size = batch_size )

Epoch 1/10
350/350 [==============================] - 69s 175ms/step - loss: 0.5286 - accuracy: 0.7622 - val_loss: 0.4982 - val_accuracy: 0.7593
Epoch 2/10
350/350 [==============================] - 61s 174ms/step - loss: 0.3905 - accuracy: 0.8351 - val_loss: 0.3947 - val_accuracy: 0.8371
Epoch 3/10
350/350 [==============================] - 60s 172ms/step - loss: 0.3204 - accuracy: 0.8624 - val_loss: 0.3958 - val_accuracy: 0.8271
Epoch 4/10
350/350 [==============================] - 60s 172ms/step - loss: 0.2884 - accuracy: 0.8786 - val_loss: 0.4143 - val_accuracy: 0.8293
Epoch 5/10
350/350 [==============================] - 61s 174ms/step - loss: 0.2513 - accuracy: 0.8958 - val_loss: 0.4269 - val_accuracy: 0.8336
Epoch 6/10
350/350 [==============================] - 60s 170ms/step - loss: 0.2176 - accuracy: 0.9121 - val_loss: 0.4785 - val_accuracy: 0.8171
Epoch 7/10
350/350 [==============================] - 60s 172ms/step - loss: 0.1924 - accuracy: 0.9238 - val_loss: 0.4675 - val_ac

In [15]:
loss, accuracy = model1.evaluate( x_train, y_train, verbose = False )
print( "\nTraining Accuracy: {:.4f}, Training Loss: {:.4f}".format(accuracy, loss) )

loss, accuracy = model1.evaluate( x_test, y_test, verbose = False )
print( "Testing Accuracy:  {:.4f}, Testing Loss: {:.4f}\n".format(accuracy, loss) )

# print(history.history)


Training Accuracy: 0.9812, Training Loss: 0.0644
Testing Accuracy:  0.7979, Testing Loss: 0.6902



In [ ]:
def gru(dense, maxlen):
    gru_out = 128

    model1 = Sequential()
    model1.add( layers.Embedding( vocab_size, embedding_dim, weights = [embedding_matrix], input_length = maxlen, trainable = True ) )
    model1.add(layers.Bidirectional(layers.GRU(gru_out, dropout=0.2)))
    model1.add( layers.Dense ( dense, activation ='sigmoid' ) )

    model1.compile ( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'] )
    print ( model1.summary() )

    return model1

model2 = gru (1, maxlen)

In [17]:
batch_size = 32
history = model2.fit( x_train, y_train, epochs = 10, verbose = True, validation_data = ( x_val, y_val ), batch_size = batch_size )

Epoch 1/10
350/350 [==============================] - 49s 129ms/step - loss: 0.5320 - accuracy: 0.7629 - val_loss: 0.4712 - val_accuracy: 0.7750
Epoch 2/10
350/350 [==============================] - 44s 125ms/step - loss: 0.3793 - accuracy: 0.8395 - val_loss: 0.4025 - val_accuracy: 0.8429
Epoch 3/10
350/350 [==============================] - 44s 127ms/step - loss: 0.3217 - accuracy: 0.8647 - val_loss: 0.3867 - val_accuracy: 0.8350
Epoch 4/10
350/350 [==============================] - 46s 132ms/step - loss: 0.2843 - accuracy: 0.8821 - val_loss: 0.4020 - val_accuracy: 0.8371
Epoch 5/10
350/350 [==============================] - 44s 126ms/step - loss: 0.2513 - accuracy: 0.8965 - val_loss: 0.4388 - val_accuracy: 0.8121
Epoch 6/10
350/350 [==============================] - 44s 124ms/step - loss: 0.2109 - accuracy: 0.9137 - val_loss: 0.4480 - val_accuracy: 0.8229
Epoch 7/10
350/350 [==============================] - 44s 127ms/step - loss: 0.1758 - accuracy: 0.9304 - val_loss: 0.5374 - val_ac

In [ ]:
loss, accuracy = model2.evaluate( x_train, y_train, verbose = False )
print( "\nTraining Accuracy: {:.4f}, Training Loss: {:.4f}".format(accuracy, loss) )

loss, accuracy = model2.evaluate( x_test, y_test, verbose = False )
print( "Testing Accuracy:  {:.4f}, Testing Loss: {:.4f}\n".format(accuracy, loss) )

# print(history.history)

In [ ]:
# Comparing the four different models
print(f"Simple LSTM model loss and accuracy: {model.evaluate(x_test, y_test)} " )
print(f"bi-LSTM model loss and accuracy: {model1.evaluate(x_test, y_test)} " )
print(f"GRU model loss and accuracy: {model2.evaluate(x_test, y_test)}")

In [23]:
max_len = 50
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
vocab_size = 3000

In [22]:
detect_msg = ["Hahaha I just wish they would live their truth and cut the shit, so we wouldn't have to worry about these mfs being out in society.",
               "just the inside of your wallet: cash, bank cards, credit cards, debit cards. That is the inside that matters to femoids."]
def detect_sexism(detect_msg):
  new_seq = tokenizer.texts_to_sequences(detect_msg)
  padded = pad_sequences(new_seq,
                         maxlen = max_len,
                         padding = padding_type,
                         truncating = trunc_type)
  return(model.predict(padded))
detect_sexism(detect_msg)

NameError: name 'model' is not defined